In [2]:
import os

In [3]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam, SGD 

2021-08-22 07:30:04.361342: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-22 07:30:04.361379: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [5]:
df = pd.read_csv('last_clean.csv',encoding="cp949")

In [6]:
df[['사고']]

,사고
0,1
1,1
2,1
3,0
4,1
...,...
28935,0
28936,0
28937,1
28938,1


In [7]:
y_data = df[['사고']]
y_data = y_data.astype(np.float32)
len(y_data)

28940

In [8]:
x_data = df.loc[:, ['유의파고','파향','최대파주기','풍속','풍향']]
x_data = x_data.astype(np.float32)
len(x_data)

28940

In [9]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

In [10]:
scaler = StandardScaler()
x_data_scaled = scaler.fit_transform(x_data) #변수들의 스케일 달라 맞춰주는 부분

In [11]:
encoder = OneHotEncoder()
y_data_encoded = encoder.fit_transform(y_data).toarray() #정답부분을 0,1로 하기위해 원핫인코딩사용

In [12]:
x_train, x_val, y_train, y_val = train_test_split(x_data_scaled, y_data_encoded, test_size=0.2, random_state=2021)
# 전체 사고+무사고 데이터를 8:2의 비율로 train과 validation 데이터셋으로 나눔


In [13]:
x_train.shape

(23152, 5)

In [14]:
y_train

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [1., 0.]])

In [15]:
y_train=y_train.astype(int)
y_train

array([[0, 1],
       [0, 1],
       [0, 1],
       ...,
       [0, 1],
       [0, 1],
       [1, 0]])

In [16]:
type(x_train[0][0])
type(y_train[0][0])

numpy.int64

In [17]:
import numpy
y_train=y_train.astype(numpy.float32)
type(y_train[0][0])

numpy.float32

In [18]:
from tensorflow.keras import optimizers
import tensorflow as tf
model = Sequential() #레이어를 선형으로 연결 구성

model.add(Dense(2,input_shape=(5,) , activation='sigmoid'))

#회귀분석이라 은닉층이 sigmoid 활성화함수를 통과하는 부분 밖에 없음

model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['acc'])
#이진분류를 할때 binary_crossentropy 사용, 
#데이터양이나 은닉층이 많아질수록 시간이 오래걸리니까 이를 고안해 경사하강법 사용
#sgd를 보완한 momentum과 adagrad의 단점을 보완한 RMSProp를 합친 Adam을 옵티마이저로 설정




2021-08-22 07:30:08.639791: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-08-22 07:30:08.639827: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-08-22 07:30:08.639851: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-172-31-5-249): /proc/driver/nvidia/version does not exist
2021-08-22 07:30:08.640088: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [19]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

early_stop=EarlyStopping(monitor='val_loss',patience=1,verbose=1)
best = ModelCheckpoint('best_model',monitor='val_acc',mode='max',verbose=1)

#최적 모델을 best에 저장
#오버피팅을 막게 early_stop을함

model.fit(
    x_train,
    y_train,
    epochs=20,
    batch_size = 3,
    verbose=1,
    validation_data=(x_val, y_val), 
    callbacks=[early_stop,best]
)

2021-08-22 07:30:08.829172: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/20
7718/7718 [==============================] - 9s 1ms/step - loss: 0.6252 - acc: 0.6752 - val_loss: 0.6082 - val_acc: 0.6923

Epoch 00001: saving model to best_model


2021-08-22 07:30:17.931651: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: best_model/assets
Epoch 2/20
7718/7718 [==============================] - 8s 1ms/step - loss: 0.5928 - acc: 0.7075 - val_loss: 0.6013 - val_acc: 0.6978

Epoch 00002: saving model to best_model
INFO:tensorflow:Assets written to: best_model/assets
Epoch 3/20
7718/7718 [==============================] - 8s 1ms/step - loss: 0.5894 - acc: 0.7107 - val_loss: 0.6003 - val_acc: 0.7006

Epoch 00003: saving model to best_model
INFO:tensorflow:Assets written to: best_model/assets
Epoch 4/20
7718/7718 [==============================] - 8s 1ms/step - loss: 0.5886 - acc: 0.7112 - val_loss: 0.6013 - val_acc: 0.6961

Epoch 00004: saving model to best_model
INFO:tensorflow:Assets written to: best_model/assets
Epoch 00004: early stopping


In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 2)                 12        
Total params: 12
Trainable params: 12
Non-trainable params: 0
_________________________________________________________________


In [21]:
from tensorflow.keras.models import load_model
loaded_model = load_model('best_model')
print('\n테스트 정확도 : %.4f'%(loaded_model.evaluate(x_val,y_val)[1]))


181/181 [==============================] - 0s 773us/step - loss: 0.6013 - acc: 0.6961

테스트 정확도 : 0.6961


In [32]:
print(loaded_model.evaluate(x_val, y_val))

[[-0.45202193  0.21590278 -0.3496556  -0.3277538  -0.09107403]
 [ 0.33554077 -0.44329712  1.7747538   0.15924236  0.3891188 ]
 [-0.25513127  1.650108    0.16177623 -0.04128544  1.3145813 ]
 ...
 [-0.8458033   0.34952438 -0.6250421  -1.3303932  -0.13472793]
 [-0.64891267 -0.71944845 -0.4676784  -1.015278    0.7645423 ]
 [-0.9442487   1.0354487  -1.136474   -0.01263863  0.45896503]]
(5,)


In [23]:

result=loaded_model.predict(x_val, verbose=1)

181/181 [==============================] - 0s 634us/step


In [24]:
import json
import time

In [25]:
result = pd.DataFrame(result)

In [26]:
result_df = pd.DataFrame({
    'trench_num':list(range(0,5788)),
    'loss':result[:][0],
    'acc' : result[:][0]
    })
result_df

,trench_num,loss,acc
0,0,0.392037,0.392037
1,1,0.684855,0.684855
2,2,0.350235,0.350235
3,3,0.627759,0.627759
4,4,0.721799,0.721799
...,...,...,...
5783,5783,0.591301,0.591301
5784,5784,0.658035,0.658035
5785,5785,0.354869,0.354869
5786,5786,0.454783,0.454783


In [27]:
result_json = result_df.to_json(orient= 'table')
ob = json.loads(result_json)
with open("result_json.json", "w") as outfile:
    json.dump(ob, outfile, sort_keys=True, indent =4)